# IMT 3860 - Introducción a Data Science

# Webscrapping (Parte 2): Requests a HTML

## 1. Requests/ Solicitudes post.

Este notebook corresponde a un ejemplo de extracción de datos de la base de datos de toc toc utilizando solicitudes 
de tipo get y de tipo post.

Para realizar la solicitud post en la aplicación de mapa que nos permite acceder a los datos, es necesario obtener una token la cual se encuentra en el texto plano lo que nos obliga a aplicar lo aprendido recientemente.

Las solicitudes POST son frecuentemente asociadas a formas y contienen parámetros de consulta que definirán que data será enviada por parte del servidor (generalmente a traves de API's)

In [ ]:
import requests
import json
import time
import re
import pandas as pd

### Obtención de token 

En este paso simplemente se extrae el HTML y se utilizan expresiones regulares para extraer la token del texto plano

In [ ]:
def get_token_casa():
    url='https://www.toctoc.com/resultados/mapa/compra/casa/'
    user_agent= 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    headers= {'User-Agent': user_agent}
    source= requests.get(url, headers)
    pattern= r'\"token":".{,}?"'
    token= re.search(pattern, source.text).group().strip('"token"').strip(":").strip('"')
    return token
def get_token_departamento():
    url='https://www.toctoc.com/resultados/mapa/compra/departamento/'
    user_agent= 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    headers= {'User-Agent': user_agent}
    source= requests.get(url, headers)
    pattern= r'\"token":".{,}?"'
    token= re.search(pattern, source.text).group().strip('"token"').strip(":").strip('"')
    return token

In [ ]:
url='https://www.toctoc.com/'
user_agent= 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
headers= {'User-Agent': user_agent}
source= requests.get(url, headers)

In [ ]:
source= requests.get(url, headers)

In [ ]:
pattern= r'\"token":".{,}?"'

In [ ]:
re.search(pattern, source.text)

In [ ]:
source

In [ ]:
get_token_departamento()

### Generación de solicitud de data

Tras analizar el sitio, se genera lo necesario para la solicitud de data utilizando la función post de la librería requests

In [ ]:
def get_toc_toc(pagina, tipo, operacion):
    
    ### operacion 1,2 (compra, arriendo), tipo: "departamento", "casa", "terreno", "oficina", "local comercial"
    url='https://www.toctoc.com/api/mapa/GetPropiedades2'
    method='POST'
    if tipo == 'casa':
        token = get_token_casa()
    if tipo == 'departamento':
        token = get_token_departamento()

    payload={ 
    'region': "",
    'comuna': "",
    'barrio': "",
    'poi': "",
    'tipoVista': "mapa",
    'operacion': operacion,
    'idPoligono': '0',
    'moneda': '2',
    'precioDesde': '0',
    'precioHasta': '0',
    'dormitoriosDesde': '0',
    'dormitoriosHasta': '0',
    'banosDesde': '0',
    'banosHasta': '0',
    'tipoPropiedad': tipo,
    'estado': '0',
    'disponibilidadEntrega': "",
    'numeroDeDiasTocToc': '0',
    'superficieDesdeUtil': '0',
    'superficieHastaUtil': '0',
    'superficieDesdeConstruida': '0',
    'superficieHastaConstruida': '0',
    'superficieDesdeTerraza': '0',
    'superficieHastaTerraza': '0',
    'superficieDesdeTerreno': '0',
    'superficieHastaTerreno': '0',
    'ordenarPor': '0',
    'pagina': '{0}'.format(pagina+1),
    'paginaInterna': '1',
    'zoom': '2.9604474313144915',
    'idZonaHomogenea': '0',
    'busqueda': "",
    'viewport': "-53.322206687377715,-94.01165015122845,-5.265176625303226,-55.80055985968981",
    'atributos': '[]',
    'publicador': '0',
    'temporalidad': '0',
    'limite': '510',
    'cargaBanner': 'false',
    'primeraCarga': 'false'}
    datareq=json.dumps(payload)
    response= requests.post(url=url, headers={
    'x-access-token': token,
    'content-type': 'application/json',
    'origin': 'https://www.toctoc.com'}, data= datareq)
    time.sleep(1.5)
    return response

### Procesamiento

De acá en adelante se generan funciones para estructurar el dato deseado para construir en csv


In [ ]:
def registro_to_dataframe(dataframe, propiedad):
    registro={'codigo': propiedad[1],
    'longitud': propiedad[2],
    'latitud': propiedad[3],
    'comuna': propiedad[7],
    'direccion': propiedad[39],
    'm21': propiedad[27],
    'm22': propiedad[28],
    'bathroom': propiedad[4],
    'dormitorios':propiedad[5],
    'precio11': propiedad[22],
    'precio12': propiedad[23],
    'precio21': propiedad[24],
    'precio22': propiedad[25],
    'uf': min(propiedad[22],propiedad[24]),
    'fecha_publicacion': propiedad[14],
    'url': propiedad[40],
    'url_img': propiedad[20],
    'operacion': 'venta'
    }
    dataframe=dataframe.append([registro])
    return dataframe

In [ ]:
#Construccion del dataframe basado en la respuesta

In [ ]:
def toc_toc_to_dataframe(tipo, operacion):
    fail_list=[]
    dataframe=pd.DataFrame(columns=['codigo', 'longitud', 'latitud', 'comuna', 'direccion', 'm21', 'm22', 'bathroom', 'dormitorios', 'precio11', 'precio12','precio21', 'precio22',
                                    'uf','fecha_publicacion', 'url', 'url_img'])
    max_paginas=10000
    for pagina in range(10000):
        print("Extrayendo pagina {0}".format(pagina))
        if max_paginas<pagina:
            break
            
        propiedades=get_toc_toc(pagina=pagina, tipo=tipo, operacion=operacion)
        
        if pagina==0:
            print(propiedades.status_code)
            if propiedades.status_code==200:
                max_paginas=round(propiedades.json()['resultados']['Total']/510)
                print(max_paginas)
            else:
                print('fallo al principio')
                dataframe=toc_toc_to_dataframe(tipo=tipo, operacion=operacion)
                return dataframe

        if propiedades.status_code==200:
            for propiedad in propiedades.json()['resultados']['Propiedades']:
                dataframe=registro_to_dataframe(dataframe=dataframe, propiedad=propiedad)
 
        else:
            print("pagina {0} no funciono".format(pagina))
            fail_list.append(pagina)
    if len(fail_list)>0:
        print(fail_list)
        dataframe=retry_to_df(fail_list=fail_list, tipo=tipo, operacion= operacion, dataframe=dataframe)
        
            

    return dataframe
    
        
        

### Control de posibles errores
El error mas común en este sitio es que solicitudes sucesivas en poco tiempo no siempre obtienen respuestas, por lo cual se genera una función recursiva que consultará los anuncios no obtenidos hasta tenerlos todos

In [ ]:
def retry_to_df(fail_list, tipo, operacion,dataframe):
    fail_list_redo=[]
    for pagina in fail_list:
        propiedades=get_toc_toc(pagina, tipo=tipo, operacion=operacion)
        if propiedades.status_code==200:
            for propiedad in propiedades.json()['resultados']['Propiedades']:
                dataframe=registro_to_dataframe(dataframe= dataframe, propiedad=propiedad)
 
        else:
            print("pagina {0} no funciono".format(pagina))
            fail_list_redo.append(pagina)
            
    if len(fail_list_redo)>0:
        retry_to_df(fail_list=fail_list_redo, tipo=tipo, operacion=operacion, dataframe=dataframe)
    return dataframe
    

In [ ]:
#Generación de csv 

In [ ]:
def gen_csv(tipo, operacion):
    df=toc_toc_to_dataframe(tipo=tipo, operacion=operacion)
    df.to_csv("{0}{1}.csv".format(tipo, operacion))

In [ ]:
gen_csv("casa", 1)

In [ ]:
gen_csv("casa", 2)

In [ ]:
gen_csv("departamento", 1)

In [ ]:
gen_csv("departamento", 2)